In [1]:
import numpy as np
import sympy as sp
from sympy.abc import x, y
import matplotlib.pyplot as plt
import pickle
from datetime import datetime
from functools import reduce
from scipy.optimize import fsolve

In [2]:
def full_soln(x):
    return list(x) + [1 - sum(x[1:])]

In [3]:
# NE_data = dict()
# for N in range(3, 5):
#     start_time = datetime.now()
#     print("N = ", N)
#     print("Start Time: ", start_time)
    
#     p = sp.IndexedBase('p')
#     i = sp.symbols('i', cls=sp.Idx, range=(1,N))
#     Z0, c0 = sp.symbols('Z0, c0')
#     Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
#     L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
#     OneMinusL = lambda i: lambda Q: Q - L(i)(Q)
#     Z = lambda i: Z0 if i==0 else reduce(lambda f,g: lambda x: f(g(x)), [OneMinusL(x) for x in range(1, i+1)])(Z0)
#     c = lambda i: Z(i-1).subs(p[i], 0)
#     func = lambda x: [sp.lambdify([c0]+[p[i] for i in range(1, N)], c(j).subs(p[N], 1-sp.summation(p[i], (i, 1, N-1))).simplify()-c0)(*x) for j in range(1, N+1)]
#     soln = full_soln(fsolve(func, [1/N for _ in range(N)]))
#     print(soln)
#     NE_data[N] = soln
#     # with open("data/NE{}.pk".format(N), 'wb') as file:
#     #     pickle.dump(soln, file)
    
#     end_time = datetime.now()
#     print("End Time: ", end_time)
#     diff = end_time - start_time
#     print("Elapsed Time: {} days {} hours {} minutes {} seconds {} milliseconds {} microseconds".format(diff.days, diff.seconds // 3600, (diff.seconds // 60) % 60, diff.seconds % 60, diff.microseconds // 1000, diff.microseconds % 1000))
#     print()

In [4]:
N = 3
p = sp.IndexedBase('p')
i,j = sp.symbols('i j', cls=sp.Idx, range=(1,N))
Z0, c0 = sp.symbols('Z0, c0')
Z0 = sp.summation(p[i], (i, 1, N))**(N-1)
L = lambda i: lambda Q: p[i]*sp.diff(Q, p[i]).subs(p[i], 0)
OneMinusL = lambda i: lambda Q: Q - L(i)(Q)
Z = lambda i: Z0 if i==0 else reduce(lambda f,g: lambda x: f(g(x)), [OneMinusL(x) for x in range(1, i+1)])(Z0)
c = lambda i: Z(i-1).subs(p[i], 0)

In [5]:
for i in range(1, N):
    display(c(i))
display(c(N))

(p[2] + p[3])**2

(p[1] + p[3])**2 - 2*p[1]*p[3]

(p[1] + p[2])**2 - 2*p[1]*p[2]

In [6]:
# c0 = 0.2
# ps = dict()
# for n in range(1, N):
#     func = c(n).subs(p[N], 1-sp.summation(p[j], (j, 1, N-1)))-c0
#     display(func)
#     func = func.subs(ps)
#     ps[p[n]] = sp.solve(func, p[n])[0]
#     display(func)
#     print(ps[p[n]])
# func = c(N).subs(p[1], 1-sp.summation(p[j], (j, 2, N)))-c0
# display(func)
# func = func.subs(ps)
# display(func)
# pN = sp.solve(func, p[N])
# print(pN)

In [12]:
ps = {c0: 0.27}
derivs = dict()
for i in range(1, N):
    ci = c(i).subs(p[N], 1 - sp.summation(p[j], (j, 1, N-1)))
    eq = ci - c0
    display(eq)
    soln = sp.solve(eq.subs(ps), p[i])
    soln = soln[0]
    ps[p[i]] = soln
    
    pdvs = dict()
    for l in range(i):
        sub = {p[i]: y, (c0 if l==0 else p[l]): x}
        inv_sub = {v:k for k,v in sub.items()}
        pdvs[l] = sp.idiff(eq.subs(sub), y, x).subs(inv_sub).subs(ps).simplify()

    derivs[i] = pdvs[0] + sum([derivs[k]*pdvs[k] for k in range(1, i)])

ps[p[N]] = 1 - sp.summation(p[j], (j, 1, N-1)).subs(ps)
strat = [ps[p[1]], ps[p[2]], ps[p[3]]]
sum(np.abs(prob) for prob in strat)
print(strat)

-c0 + (1 - p[1])**2

-c0 + (1 - p[2])**2 - 2*(-p[1] - p[2] + 1)*p[1]

[0.480384757729337, 0.321548373407327, 0.198066868863336]
1.00000000000000


In [15]:
c(N).subs(ps)

0.334162871899572